In [11]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import MaxAbsScaler

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)


In [12]:
#dataset = pd.read_pickle('SNR_BBH-injections-5000.pkl')
#dataset = pd.read_pickle('SNR_BBH-injections-reduced-100k.pkl')
dataset = pd.read_pickle('SNR_BBH-injections-reduced-1m.pkl')


dataset.head()

,ds,Mc,q,inc,spin1_mag,spin1_polar,spin1_azimuthal,spin2_mag,spin2_polar,spin2_azimuthal,coa_phase,dec,ra,pol,tc,SNR
0,1000.0,30.336864,2.002997,2.608599,0.559640,0.002940,1.788084,0.263034,0.911996,2.976439,0.0,-0.245609,3.745416,0.0,1.192530e+09,28.436068
1,1000.0,27.933059,1.063466,2.274400,0.070477,1.371662,4.080671,0.910459,0.400873,4.445331,0.0,0.252191,5.392380,0.0,1.192530e+09,9.574330
2,1000.0,19.412520,2.528205,1.988473,0.418485,1.392812,5.568183,0.264728,0.307003,2.490901,0.0,-1.131917,2.674948,0.0,1.192530e+09,11.382175
3,1000.0,21.674950,1.102297,1.628139,0.361641,0.843580,5.902105,0.606076,2.047297,1.159646,0.0,-1.227252,2.150521,0.0,1.192530e+09,5.773854
4,1000.0,38.872794,1.099620,0.809200,0.311437,0.246346,3.251720,0.399867,1.179204,3.741881,0.0,1.086481,3.558429,0.0,1.192530e+09,14.345895


In [13]:
dataset = dataset.loc[dataset["SNR"] <= 100]
dataset.shape

(999996, 16)

In [14]:
# Separate Target Variable and Predictor Variables
TargetVariable = ['SNR']
Predictors = ['Mc', 'q', 'inc', 'ra' , 'dec' , 'spin1_mag', 'spin1_polar', 'spin1_azimuthal', 'spin2_mag', 'spin2_polar', 'spin2_azimuthal']
#Predictors = ['ds', 'Mc', 'q', 'inc', 'ra' , 'dec' , 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z']

X = dataset[Predictors].values
y = dataset[TargetVariable].values
 
# Sandardization of data #
PredictorScaler = StandardScaler()
Xscaled = PredictorScaler.fit_transform(X)


TargetScaler =  StandardScaler()
yscaled = TargetScaler.fit_transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xscaled, yscaled, test_size=0.3, random_state=42)
 
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(699997, 11)
(699997, 1)
(299999, 11)
(299999, 1)


In [15]:

from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import RandomNormal
from keras.layers import PReLU
from keras.initializers import Constant

input_dim = X_train.shape[1]

# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list = [ 500, 1000, 1500, 2000]
    epoch_list  =   [ 200, 400, 800]
    
    import pandas as pd
    SearchResultsData = pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber = 0
    
    # number of neurons per layer
    
    neurons = 250
    
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber += 1
            # create ANN model
            model = Sequential()
            #kernel_initializer = RandomNormal(mean=0., stddev=0.1, seed=None)
            kernel_initializer = RandomNormal

            # Defining the Input layer and FIRST hidden layer, both are same!
            model.add(Dense(units = neurons, input_dim=input_dim, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
            model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
 
            # Defining the Second layer of the model
            # after the first layer we don't have to specify input_dim as keras configure it automatically
            model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
            model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
 
            # Defining the Third layer of the model
            # after the first layer we don't have to specify input_dim as keras configure it automatically
            model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
            model.add(PReLU(alpha_initializer=Constant(value=0.25)))

            # Defining the Fourth layer of the model
            # after the first layer we don't have to specify input_dim as keras configure it automatically
            model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
            model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
 
            # Defining the Fifth layer of the model
            # after the first layer we don't have to specify input_dim as keras configure it automatically
            model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
            model.add(PReLU(alpha_initializer=Constant(value=0.25)))

            # Defining the Sixth layer of the model
            # after the first layer we don't have to specify input_dim as keras configure it automatically
            model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
            model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
 

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer=kernel_initializer))
 
            # Compiling the model
            #model.compile(loss='mean_squared_error', optimizer='rmsprop')
            model.compile(loss='mean_squared_error', optimizer='Nadam')
            #model.compile(loss='mean_squared_error', optimizer='Adam')
            #model.compile(loss='mean_squared_error', optimizer=optimizer)


 
            # Fitting the ANN to the Training set
            model.fit(X_train, y_train, batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)
    
            # Scaling the y_test Price data back to original price scale
            y_test_orig = TargetScaler.inverse_transform(y_test)

            # Generating Predictions on testing data
            Predictions_scaled = model.predict(X_test)

            # Scaling the predicted Price data back to original price scale
            Predictions = TargetScaler.inverse_transform(Predictions_scaled)

            # Computing the absolute percent error
            APE = 100*(abs( (y_test_orig-Predictions)/y_test_orig) )
            MAPE = np.mean(APE)
            
            # printing the results of the current iteration
            print('Trial #:', TrialNumber, '  batch_size:', batch_size_trial, '  epochs:', epochs_trial, '  Accuracy:', 100-MAPE)
            
            SearchResultsData = pd.concat([SearchResultsData, pd.DataFrame(data=[[TrialNumber, int(batch_size_trial), int(epochs_trial), str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                           columns=['TrialNumber', 'Batch size', 'Epochs', 'Parameters', 'Accuracy'] ) ], ignore_index=True )
            
    return(SearchResultsData)
 

# Calling the function
ResultsData = FunctionFindBestParams(X_train, y_train, X_test, y_test)

Trial #: 1   batch_size: 500   epochs: 200   Accuracy: 96.5674026203079
Trial #: 2   batch_size: 500   epochs: 400   Accuracy: 96.76311541905734
Trial #: 3   batch_size: 500   epochs: 800   Accuracy: 96.98476370761277
Trial #: 4   batch_size: 1000   epochs: 200   Accuracy: 96.62395348635799
Trial #: 5   batch_size: 1000   epochs: 400   Accuracy: 96.58244980603631
Trial #: 6   batch_size: 1000   epochs: 800   Accuracy: 96.6298272719507
Trial #: 7   batch_size: 1500   epochs: 200   Accuracy: 96.42376081393925
Trial #: 8   batch_size: 1500   epochs: 400   Accuracy: 96.41163361940494
Trial #: 9   batch_size: 1500   epochs: 800   Accuracy: 96.33065831845582
Trial #: 10   batch_size: 2000   epochs: 200   Accuracy: 96.31410762539777
Trial #: 11   batch_size: 2000   epochs: 400   Accuracy: 96.45935929279578
Trial #: 12   batch_size: 2000   epochs: 800   Accuracy: 96.3305374234067


In [16]:
ResultsData


,TrialNumber,Parameters,Accuracy,Batch size,Epochs
0,1,500-200,96.567403,500.0,200.0
1,2,500-400,96.763115,500.0,400.0
2,3,500-800,96.984764,500.0,800.0
3,4,1000-200,96.623953,1000.0,200.0
4,5,1000-400,96.582450,1000.0,400.0
5,6,1000-800,96.629827,1000.0,800.0
6,7,1500-200,96.423761,1500.0,200.0
7,8,1500-400,96.411634,1500.0,400.0
8,9,1500-800,96.330658,1500.0,800.0
9,10,2000-200,96.314108,2000.0,200.0
